# Problem

#### which neighbourhood to live in if you are a Foodie?

## New York

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import folium

In [2]:
CLIENT_ID = 'HZZ4IK3LZYPDPVEOEY0CCQ5VRFS2EENO10MXBBBY4P240YQD' # your Foursquare ID
CLIENT_SECRET = '4EV4F1GWHABU3LY34ZN45S4U2VR3O42J1KRV3UNV2KB3FAKN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
radius = 500

In [3]:
df = pd.read_csv('new_york2.csv')

In [4]:
df.rename(columns = {'Unnamed: 0':'area'},inplace = True)

In [5]:
df.head()

,area,neighbourhood,latitude,longitude
0,Brooklyn,Kensington,40.64749,-73.97237
1,Manhattan,Midtown,40.75362,-73.98377
2,Manhattan,Harlem,40.80902,-73.94190
3,Brooklyn,Clinton Hill,40.68514,-73.95976
4,Manhattan,East Harlem,40.79851,-73.94399


In [6]:
df.shape

(48895, 4)

### Limiting size of data to 200 entries as 48895 is too much

In [7]:
df = df.sample(n=200)
df.reset_index(inplace=True)

In [8]:
df.shape

(200, 5)

In [9]:
df.head()

,index,area,neighbourhood,latitude,longitude
0,3767,Brooklyn,Greenpoint,40.72569,-73.94017
1,19915,Brooklyn,Bedford-Stuyvesant,40.68411,-73.94494
2,11768,Manhattan,West Village,40.73533,-74.00378
3,21556,Brooklyn,Bensonhurst,40.60877,-73.97382
4,45074,Manhattan,Battery Park City,40.70622,-74.01651


### Seting the center as mean of the entry points

In [10]:
latitude = df['latitude'].mean()
longitude = df['longitude'].mean()

## Map of choosen neighbourhoods

In [11]:
map_new_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['latitude'], df['longitude'], df['neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_new_york)  
    
map_new_york

In [15]:
# restaurant_types = ['Ethnic','Fast food','Fast casual','Casual dining','Premium casual','Family style','Fine dining','Cafe']

In [12]:
def getRestaurants(latitude,longitude,search_query):
    global CLIENT_ID, CLIENT_SECRET, VERSION, LIMIT, radius
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    return venues

In [14]:
# rdf = pd.DataFrame(getRestaurants(40.64749,-73.97237,'Restaurant'))
# rdf.head()
# getRestaurants(40.64749,-73.97237,'Restaurant')

def getDataFrame(s):
    names = []
    category = []
    location = []
    lat = []
    lng = []
    distance = []
#     rid = []
    for i in s:
        names.append(i['name'])
        location.append(i['location']['formattedAddress'][0])
        lat.append(i['location']['lat'])
        lng.append(i['location']['lng'])
        category.append(i['categories'][0]['shortName'])
        distance.append(i['location']['distance'])
#         rid.append(i['labeledLatLngs'][0]['refferalId'])
#     rdf = pd.DataFrame({'name':names,'category':category,'location':location,'latitude':lat,'longitude':lng,'distance':distance,'rid':rid})
    rdf = pd.DataFrame({'name':names,'category':category,'location':location,'latitude':lat,'longitude':lng,'distance':distance})
    return rdf

def getCategories(s):
    ent = []
    for i in s:
        try:
            ent.append(i['categories'][0]['shortName'])
        except:
            pass
    return ent
# getDataFrame(getRestaurants(40.64749,-73.97237,'Restaurant'))

### For variety we calculate the entropy column, which indicates variation in categories arround

In [15]:
from scipy.stats import entropy
import numpy as np

In [17]:
en = []
for i in range(0,200):
#     print(df.iloc[i,2],df.iloc[i,3])
# #     print(getRestaurants(df.iloc[i,2],df.iloc[i,3],'Restaurant')[0]['categories'][0]['shortName'])
    print(i)
    t = getCategories(getRestaurants(df.iloc[i,3],df.iloc[i,4],'Restaurant'))
    counts = pd.Series(t).value_counts()
    en.append(entropy(counts))
print(en)
# getCategories(getRestaurants(df.iloc[1,3],df.iloc[1,4],'Restaurant'))
# getRestaurants(df.iloc[1,3],df.iloc[1,4],'Restaurant')[17]
# t = getCategories(getRestaurants(df.iloc[0,3],df.iloc[0,4],'Restaurant'))
# counts = pd.Series(t).value_counts()
# entropy(counts)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
[1.559581156259877, 1.0733942812811266, 2.5631174342431216, 1.7328679513998633, 2.5459277031501344, 1.0549201679861442, 2.602380779932482, 1.9773424629430965, 2.6250481587325396, 2.6236829959546073, 2.242973226438147, 2.216280471258511, 2.512658529529802, 1.5498260458782016, 1.8914036877117695, 2.739527665685

In [19]:
print(max(en),min(en))

3.0140772805713656 0.0


In [20]:
df['variety'] = en

In [21]:
df.head()

,index,area,neighbourhood,latitude,longitude,variety
0,3767,Brooklyn,Greenpoint,40.72569,-73.94017,1.559581
1,19915,Brooklyn,Bedford-Stuyvesant,40.68411,-73.94494,1.073394
2,11768,Manhattan,West Village,40.73533,-74.00378,2.563117
3,21556,Brooklyn,Bensonhurst,40.60877,-73.97382,1.732868
4,45074,Manhattan,Battery Park City,40.70622,-74.01651,2.545928


## Map plot depicting food variety choices

bigger circle implies more variety

In [25]:
food_choices = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
i=0
for lat, lng, label in zip(df['latitude'], df['longitude'], df['neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 2.5*df.iloc[i,5] + 1,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(food_choices) 
    i = i + 1
    
food_choices

# Results

choosen neighborhoods
<img src="neighborhoods.png">
link = https://github.com/Akshay-s-raut/Coursera_Capstone/blob/master/neighborhoods.PNG

variety of neighborhoods  
<img src="varieties.png">
link = https://github.com/Akshay-s-raut/Coursera_Capstone/blob/master/varieties.PNG